In [1]:
import argparse
import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import tiktoken
from tree_sitter import Language, Parser

In [9]:
from characterize import BenchmarkParser

In [3]:
language = Language(os.path.join('tree_sitter_lib/build/') + 'c-tree-sitter.so', "c")

In [148]:
codefile = open('original_benchmarks/LinearArbitrary-SeaHorn/pie/ICE/benchmarks/form22.c', 'r')

In [149]:
code = codefile.read()

In [11]:
bp = BenchmarkParser(language_name="c")

In [150]:
ast = bp.parse(code)

In [151]:
ast.root_node.sexp()

'(translation_unit (declaration (storage_class_specifier) type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list) (attribute_specifier (argument_list (identifier))))) (declaration (storage_class_specifier) type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list))) (function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list (parameter_declaration type: (primitive_type) declarator: (identifier)))) body: (compound_statement (if_statement condition: (parenthesized_expression (unary_expression argument: (parenthesized_expression (identifier)))) consequence: (compound_statement (labeled_statement label: (statement_identifier) (expression_statement (call_expression function: (identifier) arguments: (argument_list)))) (expression_statement))) (return_statement))) (expression_statement (call_expression function: (identif

In [156]:
print(ast.root_node.sexp())

(translation_unit (declaration (storage_class_specifier) type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list) (attribute_specifier (argument_list (identifier))))) (declaration (storage_class_specifier) type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list))) (function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list (parameter_declaration type: (primitive_type) declarator: (identifier)))) body: (compound_statement (if_statement condition: (parenthesized_expression (unary_expression argument: (parenthesized_expression (identifier)))) consequence: (compound_statement (labeled_statement label: (statement_identifier) (expression_statement (call_expression function: (identifier) arguments: (argument_list)))) (expression_statement))) (return_statement))) (expression_statement (call_expression function: (identifi

In [158]:
query = language.query("""(((compound_statement (declaration ((identifier) @name))) @function))""")
fs = query.captures(ast.root_node)
for f in fs:
    print("=========================================================")
    print(f[0].text.decode('utf-8'))
    print(f[0].sexp())
    print(f[0].parent.text)
    print("=========================================================")

{
  int x1,x2,x3;
  int x1p,x2p,x3p, input;

  x1 = x2 = x3 = 0;
  input = __VERIFIER_nondet_int();
  while(input)
  {
    x1p = __VERIFIER_nondet_int();
    x2p = __VERIFIER_nondet_int();
    x3p = __VERIFIER_nondet_int();

    if (x1p <= x2p && (x2p >= 0 || x2p - x3p <= 2))
    {
	x1 = x1p;
	x2 = x2p;
	x3 = x3p;
    }
    input = __VERIFIER_nondet_int();
  }
  __VERIFIER_assert(x1 <= x2 && (x2 >= 0 || x2 - x3 <= 2));    
}
(compound_statement (declaration type: (primitive_type) declarator: (identifier) declarator: (identifier) declarator: (identifier)) (declaration type: (primitive_type) declarator: (identifier) declarator: (identifier) declarator: (identifier) declarator: (identifier)) (expression_statement (assignment_expression left: (identifier) right: (assignment_expression left: (identifier) right: (assignment_expression left: (identifier) right: (number_literal))))) (expression_statement (assignment_expression left: (identifier) right: (call_expression function: (identifier) a

In [104]:
query = language.query("""(function_definition) @function""")
fs = query.captures(ast.root_node)
fs

[(<Node type=function_definition, start_point=(0, 0), end_point=(5, 1)>,
  'function')]

In [100]:
q1 = language.query("""(function_declarator (identifier) @name)""").captures(fs[0][0])

In [95]:
q2 = language.query("""((identifier) @name (#not-match? @name "(__VERIFIER_.+|unknown\d)"))""").captures(q1[0][0])

'(function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (identifier)) (return_statement (identifier))))'

In [ ]:
for node in fs:
    q = language.query("""(_ ((function_declarator (identifier) @name)) (#not-match? @name "(__VERIFIER_.+|unknown\d)"))""").captures(node[0])
    if len(q) > 0:
        print(node[0].sexp())
        print(q[0][0].start_point, q[0][0].end_point, q[0][0].text)

(function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (identifier)) (return_statement (identifier))))
(0, 4) (0, 12) b'unknown1'
(function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (call_expression function: (identifier) arguments: (argument_list)))) (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (number_literal))) (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (number_literal))) (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (number_literal))) (declaration type: (primitive_type) declarator

(function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (identifier)) (return_statement (identifier))))
(0, 4) (0, 12) b'unknown1'
(function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (call_expression function: (identifier) arguments: (argument_list)))) (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (number_literal))) (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (number_literal))) (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (number_literal))) (declaration type: (primitive_type) declarator

In [39]:

ast.root_node.sexp()

'(translation_unit (function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (identifier)) (return_statement (identifier)))) (declaration type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list))) (declaration type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list))) (declaration type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list))) (function_definition type: (primitive_type) declarator: (function_declarator declarator: (identifier) parameters: (parameter_list)) body: (compound_statement (declaration type: (primitive_type) declarator: (init_declarator declarator: (identifier) value: (call_expression function: (identifier) arguments: (argument_list)))) (declaration type: (primiti